In [1]:
import tensorflow as tf
import os
import numpy as np

print(tf.__version__)

2.0.0


In [2]:
# 判断是否gpu可用,如果可用设置gpu使用显存
if tf.test.is_gpu_available():
    gpus = tf.config.experimental.list_physical_devices('GPU')
    for gpu in gpus:
        tf.config.experimental.set_memory_growth(gpu,True)

#### 周杰伦歌词数据:
链接: https://pan.baidu.com/s/1QieFe3iuDlDeyTYe4dRySA 提取码: vwvs

In [3]:
#读取数据
import random
import zipfile

with zipfile.ZipFile('./data/jaychou_lyrics.txt.zip') as zin:
    with zin.open('jaychou_lyrics.txt') as f:
        corpus_chars = f.read().decode('utf-8')

In [4]:
corpus_chars = corpus_chars.replace('\n', ' ').replace('\r', ' ')
corpus_chars = corpus_chars[0:10000]

In [5]:
# 建立字符索引
idx_to_char = list(set(corpus_chars))
char_to_idx = dict([(char, i) for i, char in enumerate(idx_to_char)])
vocab_size = len(char_to_idx)

In [6]:
corpus_indices = [char_to_idx[char] for char in corpus_chars]
sample = corpus_indices[:20]
print('chars:', ''.join([idx_to_char[idx] for idx in sample]))
print('indices:', sample)

chars: 想要有直升机 想要和你飞到宇宙去 想要和
indices: [704, 718, 433, 786, 122, 851, 25, 704, 718, 518, 201, 171, 1017, 664, 694, 220, 25, 704, 718, 518]


In [7]:
# one-hot向量
tf.one_hot(np.array([0, 2]), vocab_size)


<tf.Tensor: id=4, shape=(2, 1027), dtype=float32, numpy=
array([[1., 0., 0., ..., 0., 0., 0.],
       [0., 0., 1., ..., 0., 0., 0.]], dtype=float32)>

In [8]:
def to_onehot(X, size): 
    # X shape: (batch), output shape: (batch, n_class)
    return [tf.one_hot(x, size,dtype=tf.float32) for x in X.T]
X = np.arange(10).reshape((2, 5))
inputs = to_onehot(X, vocab_size)
len(inputs), inputs[0].shape

(5, TensorShape([2, 1027]))

In [9]:
# 从零实现 rnn
# 初始化模型参数，我们依据第二章的结论，使用 Ht = theta(concate(Xt + Ht-1, axis=-1) * W),所以可训练的模型参数是 W，输出矩阵Wo及bh,bo

# 初始化参数有：Ht-1 的维度
vocab_size = 1027
num_inputs, num_hiddens, num_outputs = vocab_size, 256, vocab_size
class Rnn(object):
    def __init__(self, hiden_dim, params=None):
        """
        初始化待训练的参数 variabel
        params = (hidden_weight, weight_out, bn_h, bn_o)
        """
        # 是否加载预训练参数
        if params:
            self.hidden_weight, self.weight_out, self.bn_h, self.bn_o = params
        else:
            self.hidden_weight = self._ones(shape=(vocab_size+ num_hiddens, num_hiddens))
            self.weight_out = self._ones(shape=(num_hiddens, vocab_size))
            self.bn_h = tf.Variable(tf.zeros([1,num_hiddens]), dtype=tf.float32)
            self.bn_o = tf.Variable(tf.zeros([1,vocab_size]), dtype=tf.float32)
        
    def _ones(self,shape):
        return tf.Variable(tf.random.normal(shape=shape,stddev=0.01,mean=0,dtype=tf.float32))
    
    def net(self,inputs,Ht):
        # 展开做循环计算
        outputs = []
        for X in inputs:
            x = tf.reshape(X,(-1, vocab_size))
            # Ht = tf.tanh(theta(concate(Xt + Ht-1, axis=-1) * W) + bn)
            Ht = tf.tanh(tf.matmul(tf.concat([X,Ht], axis=1), self.hidden_weight) + self.bn_h)
            # 计算输出
            Y= tf.matmul(Ht,self.weight_out) + self.bn_o
            # 存储中间Y的输出,一般的rnn这里设置return_sequences和return_state参数, 
            # return_sequence=True返回多个序列,return_state=True代表返回隐向量
            outputs.append(Y)

        return outputs, Ht

In [10]:
# 从零实现 GRU，其实与 RNN 的主要区别在于有门的设计
# 确定哪些是需要初始化模型参数：
# 核心公式就是一下 四个
# Rt=σ(XtWxr+Ht−1Whr+br), Zt=σ(XtWxz+Ht−1Whz+bz)， H~t=tanh(XtWxh+(Rt⊙Ht−1)Whh+bh)， Ht=Zt⊙Ht−1+(1−Zt)⊙H~t

# 初始化参数有：Ht-1 的维度
vocab_size = 1027
num_inputs, num_hiddens, num_outputs = vocab_size, 256, vocab_size
class GRU(object):
    def __init__(self, hiden_dim, params=None):
        """
        初始化待训练的参数 variabel
        params = (Wxr, Whr, br, Wxz, Whz, bz, Wxh, Whh, bh)
        """
        # 是否加载预训练参数
        if params:
            self.Wxr, self.Whr, self.br, self.Wxz, self.Whz, self.bz, self.Wxh, self.Whh, self.bh = params
        else:
            self.Wxr = self._ones(shape=(vocab_size, num_hiddens))
            self.Whr = self._ones(shape=(num_hiddens, num_hiddens))
            self.Wxz = self._ones(shape=(vocab_size, num_hiddens))
            self.Whz = self._ones(shape=(num_hiddens, num_hiddens))
            self.Wxh = self._ones(shape=(vocab_size, num_hiddens))
            self.Whh = self._ones(shape=(num_hiddens, num_hiddens))

            
            self.br = tf.Variable(tf.zeros([1,num_hiddens]), dtype=tf.float32)
            self.bz = tf.Variable(tf.zeros([1,num_hiddens]), dtype=tf.float32)
            self.bh = tf.Variable(tf.zeros([1,num_hiddens]), dtype=tf.float32)
            
            self.Whq = self._ones(shape=(num_hiddens, vocab_size))
            self.bq = tf.Variable(tf.zeros([1,vocab_size]), dtype=tf.float32)
            
    def _ones(self,shape):
        return tf.Variable(tf.random.normal(shape=shape,stddev=0.01,mean=0,dtype=tf.float32))
    
    def net(self, inputs, Ht):
        # 展开做循环计算
        outputs = []
        for X in inputs:
            x = tf.reshape(X,(-1, vocab_size))
            # Rt=σ(XtWxr+Ht−1Whr+br)
            Rt = tf.sigmoid(tf.matmul(x, self.Wxr) + tf.matmul(Ht, self.Whr) + self.br)
            # Zt=σ(XtWxz+Ht−1Whz+bz)
#             Zt = tf.sigmoid(tf.matmul(x, self.Wxz) + tf.matmul(Ht, self.Whz) + self.bz) 也开始用@用作矩阵乘法
            Zt = tf.sigmoid(x @ self.Wxz + Ht @ self.Whz + self.bz)
            # 计算 H~t=tanh(XtWxh+(Rt⊙Ht−1)Whh+bh)，注意这里的Rt和Zt都是 num_hiddens*num_hiddens 维，才能这样操作
#             H_hat = tf.tanh(tf.matmul(x,self.Wxh) + tf.multiply(Rt,Ht)@self.Whh + self.bh)
            H_hat = tf.tanh(x@self.Wxh + (Rt * Ht)@self.Whh + self.bh)
            # Ht=Zt⊙Ht−1+(1−Zt)⊙H~t 
            Ht = Zt * Ht + (1-Zt) * H_hat
            Y = tf.matmul(Ht, self.Whq) + self.bq
            outputs.append(Y)

        return outputs, Ht

In [11]:
# 初始化一个 hidden_state
def init_rnn_state(batch_size, num_hiddens):
    #随机初始化一个初始值
    return tf.zeros(shape=(batch_size, num_hiddens))

In [12]:
# 测试
state = init_rnn_state(X.shape[0], num_hiddens)
print(state.shape)
inputs = to_onehot(X, vocab_size)
print(len(inputs))# 5
print(inputs[0].shape)
gru = GRU(num_hiddens)
outputs, state_new = gru.net(inputs, state)
# print(outputs)
print(len(outputs), outputs[0].shape, state_new.shape)

(2, 256)
5
(2, 1027)
5 (2, 1027) (2, 256)


#### 定义预测函数
以下函数基于前缀prefix（含有数个字符的字符串）来预测接下来的num_chars个字符。这个函数稍显复杂，其中我们将循环神经单元rnn设置成了函数参数，这样在后面小节介绍其他循环神经网络时能重复使用这个函数。


In [13]:
# 函数说明：用来生成指定长度的字符，为了给后续模块使用，这里将 rnn模型 设为参数传入
def predict_rnn(prefix, num_chars, rnn, init_rnn_state,
                num_hiddens, vocab_size,idx_to_char, char_to_idx):
    """
    prefix: 前缀
    num_chars: 生成的字符长度
    rnn: rnn模型类
    init_rnn_state: 隐变量初始化函数
    num_hiddens: 隐变量维度
    vocab_size: 词表大小
    """
    # 初始化 rnn
    rnn = rnn(num_hiddens)
    # 初始化隐向量
    state = init_rnn_state(1, num_hiddens)
    # 生成输出字符列表,并将首字符生成
    outputs = [char_to_idx[prefix[0]]]
    # 循环解码
    for t in range(num_chars + len(prefix) -1):
        X = tf.convert_to_tensor(to_onehot(np.array([outputs[-1]]), vocab_size),dtype=tf.float32)
        X = tf.reshape(X, [1, -1])
#         print(X.shape)  # (1, 1027)
        Y, state_new = rnn.net([X], state)
        # 添加输出到 outputs 中
        if t < len(prefix) -1:
            outputs.append(char_to_idx[prefix[t + 1]])
        else:
            outputs.append(int(np.array(tf.argmax(Y[0],axis=-1))))
    print('outputs', outputs)
    return ''.join([idx_to_char[i] for i in outputs])

In [14]:
# 简单测试一下
print(predict_rnn('分开', 10, GRU, init_rnn_state, num_hiddens, vocab_size,
            idx_to_char, char_to_idx))

outputs [174, 283, 923, 734, 723, 398, 1020, 967, 1025, 364, 743, 930]
分开好习碗甩加小纳扫节涯


#### 循环神经网络中较容易出现梯度衰减或梯度爆炸。我们会在6.6节（通过时间反向传播）中解释原因。为了应对梯度爆炸，我们可以裁剪梯度（clip gradient）。假设我们把所有模型参数梯度的元素拼接成一个向量 gg，并设裁剪的阈值是θθ。裁剪后的梯度的L2范数不超过θ
min((θ / ∥g∥),1)g

In [15]:
def grad_clipping(grads,theta):
    norm = np.array([0])
    for i in range(len(grads)):
        norm+=tf.math.reduce_sum(grads[i] ** 2)
    #print("norm",norm)
    norm = np.sqrt(norm).item()
    new_gradient=[]
    if norm > theta:
        for grad in grads:
            new_gradient.append(grad * theta / norm)
    else:
        for grad in grads:
            new_gradient.append(grad)  
    #print("new_gradient",new_gradient)
    return new_gradient

#### 定义模型训练函数，跟之前章节的模型训练函数相比，这里的模型训练函数有以下几点不同：
1、使用困惑度评价模型。
2、在迭代模型参数前裁剪梯度。4